# Advanced usages

Let's create a network combining a MV part and a LV part.

In [1]:
import numpy as np
from roseau.load_flow import *

## Creation of the network

This network contains a voltage source with a constant balanced voltage of
20 kV (phase to phase), a Delta-Wye transformer and a small LV network.


In [2]:
ground = Ground()
pref = PotentialRef(ground)  # Set the ground to 0V

# Create a MV voltage source
un = 20e3 / np.sqrt(3)  # V
source_voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", n=4, ground=ground, source_voltages=source_voltages)

# Add a transformer
bus0 = Bus(id="bus0", n=4)
tc = TransformerCharacteristics(
    "160_kVA", "Dyn11", sn=160.0 * 1e3, uhv=20e3, ulv=400.0, i0=2.3 / 100, p0=460.0, psc=2350.0, vsc=4.0 / 100
)
transformer = DeltaWyeTransformer(id="transfo", bus1=vs, bus2=bus0, transformer_characteristics=tc, tap=1.025)
ground.connect(bus0)

# Add a LV network
lc = LineCharacteristics.from_name_lv("S_AL_150")
bus1 = Bus(id="bus1", n=4)
bus2 = Bus(id="bus2", n=4)
load_bus1 = Bus(id="load_bus1", n=4)
load_bus2 = Bus(id="load_bus2", n=4)
load_bus3 = Bus(id="load_bus3", n=4)
line1 = ShuntLine(id="line1", n=4, bus1=bus0, bus2=bus1, ground=ground, line_characteristics=lc, length=0.5)  # km
line2 = ShuntLine(id="line2", n=4, bus1=bus1, bus2=bus2, ground=ground, line_characteristics=lc, length=0.4)
line3 = ShuntLine(id="line3", n=4, bus1=bus1, bus2=load_bus1, ground=ground, line_characteristics=lc, length=0.3)
line4 = ShuntLine(id="line4", n=4, bus1=bus2, bus2=load_bus2, ground=ground, line_characteristics=lc, length=0.3)
line5 = ShuntLine(id="line5", n=4, bus1=load_bus2, bus2=load_bus3, ground=ground, line_characteristics=lc, length=0.4)
si = -3e3  # VA, negative as it is production
load1 = PowerLoad(id="load1", n=4, bus=load_bus1, s=[si, si, si])
load2 = PowerLoad(id="load2", n=4, bus=load_bus2, s=[si, si, si])
load3 = PowerLoad(id="load3", n=4, bus=load_bus3, s=[si, 0, 0])

# Create the network
en = ElectricalNetwork.from_element(vs)

An authentification is required. Please contact us at [contact@roseautechnologies.com](mailto:contact@roseautechnologies.com) to get identification.

In [5]:
# Authentication
auth = ("username", "password")

Then, the load flow can be solved and the voltages can be checked.

In [6]:
en.solve_load_flow(auth=auth)
np.abs(load_bus3.voltages)

Magnitude,[249.16095029692707 237.94587246515803 239.25521250335444]
Units,volt


## Usage of the flexible loads

The flexible loads are loads that implement some basic controls such as $P(U)$, $Q(U)$ or $PQ(U)$.

### $P(U)$ control

Let's remove the `load3` from the network and add a flexible load as a replacement.

We first create a `FlexibleParameter` from the class-method `p_max_u_production`. It retrieves a flexible parameter
instance that starts reducing the active production when the voltage is higher than `u_up` volts and stops the
production when the voltage reaches `u_max`. The `s_max` argument defines the maximum allowed apparent power of the
production plant. In the example below, `u_up=240 V`, `u_max=250 V` and `s_max=4 kVA`.

Then, a `FlexibleLoad` instance is created. Its apparent power is fixed at `[si, 0, 0]` VA with `si` a negative value
(negative because it is production). Theses apparent powers define the maximum power this load can produce. The
`parameters` argument takes a `FlexibleParameter` per phase. For the first phase, the previously $P(U)$ control is
used. For the two other phases, there is no control at all thus the `constant` class-method is used.

As a consequence, the provided apparent power for phase `a` is a maximum that can be produced (potentially modified by
the $P(U)$  control) and the provided apparent power for phases `b` and `c` is the desired production as the flexible
 parameter is defined as `constant`.

In [7]:
# Let's make the load 3 flexible with a p(u) control to reduce the voltages constraints
en.remove_element("load3")
parameter = FlexibleParameter.p_max_u_production(u_up=240, u_max=250, s_max=4000)  # V and VA
flexible_load = FlexibleLoad(
    id="load3",
    n=4,
    bus=load_bus3,
    s=[si, 0, 0],  # W
    parameters=[parameter, FlexibleParameter.constant(), FlexibleParameter.constant()],
)
en.add_element(flexible_load)

The load flow can now be solved again. You can see that the voltage norm has changed. Note that the voltage norm for
phase `a` was above 240 V without the $P(U)$ control, thus the control has been activated in this simulation.

In [8]:
en.solve_load_flow(auth=auth)
np.abs(load_bus3.voltages)

Magnitude,[245.893496556741 239.36821845653958 239.7046492020196]
Units,volt


The actually produced power of the flexible load is a result of the computation and can be accessed using the
`powers` property of the flexible load. Here, one can remarks that:
 * The active power for the phase `a` is negative meaning production;
 * The actual value of this active power is lower that the one requested as the control was activated;
 * The power for phases `b` and `c` is 0 VA as expected.

In [9]:
flexible_load.powers

Magnitude,[-1628.91699011+0.j 0. +0.j 0. +0.j]
Units,volt_ampere


### $PQ(U)$ control

Now, let's remove the flexible load that we have added in the previous section and add a new flexible load
implementing a $PQ(U)$ control.

As before, we first create a `FlexibleParameter` but this time, we will use the `pq_u_production` class-method. It
requires several arguments:
* `up_up` and `up_max` which are the voltages defining the interval of the `P(U)` control activation. Below `up_up`, no
 control is applied and above `u_max`, there is no production any more.
* `uq_min`, `uq_down`, `uq_up` and `uq_max` which are the voltages defining the `Q(U)` control activation.
  * Below `uq_min`, the power plant produces the maximum possible reactive power.
  * Between `uq_down` and `uq_up`, there is no `Q(U)` control.
  * Above `uq_max`, the power plant consumes the maximum possible reactive power.

In the example below, as the new load is a production load, only the `up_up`, `up_max`, `uq_up` and `uq_max` are of
interests. The $Q(U)$ control starts at 235 V and is fully used at 240 V. Then, the $P(U)$ starts at 240 V and is
fully used at 250 V.


In [10]:
# Let's try with pq(u) control, by injecting reactive power before reducing active power
en.remove_element("load3")
parameter = FlexibleParameter.pq_u_production(
    up_up=240, up_max=250, uq_min=200, uq_down=210, uq_up=235, uq_max=240, s_max=4000  # V and VA
)
flexible_load = FlexibleLoad(
    id="load3",
    n=4,
    bus=load_bus3,
    s=[si, 0, 0],
    parameters=[parameter, FlexibleParameter.constant(), FlexibleParameter.constant()],
)
en.add_element(flexible_load)

The load flow can be solved again.

In [11]:
en.solve_load_flow(auth=auth)
np.abs(load_bus3.voltages)

Magnitude,[243.55876278488515 236.2481626494373 243.80142803493553]
Units,volt


One can remark that this time, the phase `a` consumes reactive power to limit the voltage raise of the network.
Moreover, the norm of the power on phase `a` is approximately 4 kVA which is the maximum allowed apparent power for the
 `load3`. In order to maintain this maximum, a Euclidean projection has been used.

In [12]:
flexible_load.powers

Magnitude,[-2076.69942528+3418.67217664j 0. +0.j 0. +0.j ]
Units,volt_ampere


In [13]:
np.abs(flexible_load.powers)

Magnitude,[3999.999994283953 0.0 0.0]
Units,volt_ampere
